In [ ]:
import spacy
import pandas as pd

import unidecode 


from spacy.lang.nl import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest



In [ ]:
spacy.prefer_gpu

In [ ]:
df=pd.read_csv("data_03.csv",sep="|")

In [ ]:
df.shape

In [ ]:
nlp = spacy.load("nl_core_news_lg")

In [ ]:
df.iloc[780:]

In [ ]:
df.iat[0,6]

In [186]:
#LOWER CASE SEARCH 
#LEMMA THE KEYWORDS
#LEMMA THE ARTICLE

nl_tax_keywords = ["Aanslagjaar","successierechten","Wetboek der Successierechten","fiscale hervorming","fiscale gunstregime","belasting over de toegevoegde waarde",
        "BTW","kadastraal inkomen","onroerende voorheffing","Algemene Administratie van de Fiscaliteit","Inkomstenbelastingen","belastingvermindering","fiscale stimulans",
        "belastbaar tijdperk","Belcotax-on-web","Bedrijfsvoorheffing","Het Wetboek van de inkomstenbelastingen","het Wetboek diverse rechten en taksen",
        "fiscale schuldvorderingen","het Wetboek diverse rechten en taksen","fiscale schuldvorderingen","belastingen op beroepsinkomsten","vennootschapsbelastingen",
        "belastingen op onroerende goederen","verbruiksbelastingen","verkeersbelastingen","afvalbelastingen","provinciebelastingen","milieutaksen","belastingaangifte",
        "belastingteruggave","Biztax","Intervat","Rv-on-web","FinProf","MyMinFinPro","roerende voorheffing","vennootschapsbelasting","rechtspersonenbelasting","belastingkorting", 
        "Bronbelasting", "fiscale bepalingenbelastingontduiking","Belastingontwijking","fiscaal voordeel","Aftrekbare besteding","Belastingvrije som", "Belastingkrediet",
        "Decumul","Forfaitaire beroepskosten","Gemeenschappelijke aanslag","Gemiddelde aanslagvoet","Huwelijksquotiënt","Inkomstenjaar","Marginale aanslagvoet",
        "Onroerende voorheffing","Opcentiemen","Progressieve belastingtarieven","Verbeterd gemiddeld tarief","Vrijstelling met progressievoorbehoud"]





In [ ]:
text=df.iat[0,5]


doc = nlp(text)
type(doc)
len(list(doc.sents))

In [ ]:
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

In [ ]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text

In [ ]:
def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text

In [154]:
def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

In [158]:
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text

In [151]:
l=df['nl_body']

In [176]:
import re
import string

documents_clean = []
for d in l[1:2]:
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', d)
    


    document_test=accented_characters_removal(d)
    document_test=remove_newlines_tabs(document_test)
    document_test=remove_whitespace(document_test)
    document_test = document_test.lower()
    document_test=reducing_incorrect_character_repeatation(document_test)
    document_test=removing_special_characters(document_test)


    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    
    
    # Remove punctuations
    #document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
 #   document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
#    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)
    print(d)


FEDERALE OVERHEIDSDIENST BINNENLANDSE ZAKEN 10 DECEMBER 2017. - Koninklijk besluit tot wijziging van het koninklijk besluit nr. 20 van 20 juli 1970 tot vaststelling van de tarieven van de belasting over de toegevoegde waarde en tot indeling van de goederen en de diensten bij die tarieven wat bepaalde producten bestemd voor de intieme hygiënische bescherming en de externe defibrillatoren betreft. - Duitse vertaling De hierna volgende tekst is de Duitse vertaling van het koninklijk besluit van 10 december 2017 tot wijziging van het koninklijk besluit nr. 20 van 20 juli 1970 tot vaststelling van de tarieven van de belasting over de toegevoegde waarde en tot indeling van de goederen en de diensten bij die tarieven wat bepaalde producten bestemd voor de intieme hygiënische bescherming en de externe defibrillatoren betreft (Belgisch Staatsblad van 22 december 2017), bekrachtigd bij de wet van 30 juli 2018 (Belgisch Staatsblad van 10 augustus 2018, err. van 28 augustus 2018). Deze vertaling i

In [181]:
text=documents_clean[0]
text

'federale overheidsdienst binnenlandse zaken december . koninklijk besluit tot wijziging van het koninklijk besluit nr. van juli tot vaststelling van de tarieven van de belasting over de toegevoegde waarde en tot indeling van de goederen en de diensten bij die tarieven wat bepaalde producten bestemd voor de intieme hygienische bescherming en de externe defibrillatoren betreft. duitse vertaling de hierna volgende tekst is de duitse vertaling van het koninklijk besluit van december tot wijziging van het koninklijk besluit nr. van juli tot vaststelling van de tarieven van de belasting over de toegevoegde waarde en tot indeling van de goederen en de diensten bij die tarieven wat bepaalde producten bestemd voor de intieme hygienische bescherming en de externe defibrillatoren betreft (belgisch staatsblad van december ) , bekrachtigd bij de wet van juli (belgisch staatsblad van augustus , err. van augustus ) . deze vertaling is opgemaakt door de centrale dienst voor duitse vertaling in malmed

In [182]:
doc = nlp(text)
type(doc)
len(list(doc.sents))

51

In [ ]:
df.iat[0,6]

In [ ]:
df.to_csv("data_04.csv",sep="|",index=False,encoding='utf-8')

In [ ]:
documents_clean[1]

In [183]:
keyword=[]
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):   # check lemma
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.lemma_)

In [184]:
freq_word=Counter(keyword)
freq_word.most_common(10)

[('vom', 7),
 ('juli', 6),
 ('und', 5),
 ('koninklijk', 4),
 ('besluit', 4),
 ('tarief', 4),
 ('dienst', 4),
 ('extern', 4),
 ('defibrillator', 4),
 ('vertaling', 4)]

In [175]:
max_freq=Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word]=(freq_word[word]/max_freq)
freq_word.most_common(5)

[('artikel', 1.0),
 ('besluit', 0.9),
 ('familiaal', 0.8),
 ('bedoelen', 0.7),
 ('Brussels', 0.6)]

In [171]:
sent_strenght={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strenght.keys():
                sent_strenght[sent]+=freq_word[word.text]
            else:
                sent_strenght[sent]=freq_word[word.text]
print(sent_strenght)

{brussels hoofdstedelijk gewest januari .: 0.9, ministerieel besluit tot vaststelling van de modellen van formulier bedoeld in artikel , ss van het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen de minister van de brusselse hoofdstedelijke regering, belast met financien en begroting, gelet op het wetboek der successierechten, artikel bis , ss , ingevoegd door de ordonnantie van december houdende het tweede deel van de fiscale hervorming gelet op het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen, artikel gelet op de gelijkekansentest uitgevoerd in toepassing van artikel van de ordonnantie van oktober tot invoering van de gelijkekans

In [172]:
summarized_sentence=nlargest(3,sent_strenght,key=sent_strenght.get)
print(summarized_sentence)

[ministerieel besluit tot vaststelling van de modellen van formulier bedoeld in artikel , ss van het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen de minister van de brusselse hoofdstedelijke regering, belast met financien en begroting, gelet op het wetboek der successierechten, artikel bis , ss , ingevoegd door de ordonnantie van december houdende het tweede deel van de fiscale hervorming gelet op het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen, artikel gelet op de gelijkekansentest uitgevoerd in toepassing van artikel van de ordonnantie van oktober tot invoering van de gelijkekansentest overwegende dat artikel van bovengenoemd

In [ ]:
print(summarized_sentence[0])

In [ ]:
for sent in doc.sents:
    print(sent.text)
    print('-------------------------------------')

In [ ]:
text

In [ ]:
for token in doc:
    print(token.text, " / ", token.lemma_, )

In [ ]:
doc